# Plot N cross-section with matplotlib
Plots a cross-section along the length

hvPlot is the way to go, but this helped me figure out how it should work for hvPlot

In [26]:
%matplotlib widget

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from JulesD3D.processNetCDF import addDepth, makeVelocity # why make instead of addvelocity
from JulesD3D.utils import folderPicker, ncFilePicker, easyTimeFormat
import numpy as np
from os import path
from IPython.display import Markdown as md

In [5]:
parent_folder = '/Users/julesblom/ThesisResults/'
# parent_folder = '/Volumes/Seagate Expansion Drive/FinalResults/'
scenario_folder = folderPicker(parent_folder)
scenario_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom…

In [9]:
nc_filename = ncFilePicker(parent_folder)#scenario_folder.value)
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [28]:
trim = xr.open_dataset(nc_filename.value)

In [29]:
basename = path.basename(nc_filename.value)
plot_title, _ = path.splitext(basename)
plot_title

'Slope125_Run01'

In [30]:
scenario = 'Slope 1.00°'

In [31]:
# only if the NetCDF has already been processed
if "depth_center" not in trim:
    print("Need to processNetCDF first")
    trim = addDepth(trim)
    trim = makeVelocity(trim)

Need to processNetCDF first


/Users/julesblom/Library/Caches/pypoetry/virtualenvs/delft3d-python-j5EFwY34-py3.7/lib/python3.7/site-packages/xarray/core/computation.py:1215: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  return result.transpose(*[d for d in all_dims if d in result.dims])


In [32]:
SAND = 0
SILT = 1

In [33]:
md(f"""
Property  | Max Value
--------- | -------------
Density   | {float(trim.RHO.max().values)}
Velocity  | {float(trim.velocity.max().values)}
Sand conc | {float(trim.R1.isel(LSTSCI=SAND).max().values)}
Silt conc | {float(trim.R1.isel(LSTSCI=SILT).max().values)}

""")


Property  | Max Value
--------- | -------------
Density   | 1099.4537353515625
Velocity  | 3.8808841705322266
Sand conc | 34.09403610229492
Silt conc | 87.74174499511719



In [34]:
depth_center = trim['depth_center']

In [35]:
selected_time = 1
selected_M = int(trim.M.size/2)
timestring = easyTimeFormat(trim.time.isel(time=selected_time).values)

# turbu_energy_at_time = trim.RTUR1.isel(LTUR=0).isel(time=selected_time, M=selected_M)
concentration_sand = trim.R1.isel(LSTSCI=SAND, time=selected_time, M=selected_M)
concentration_silt = trim.R1.isel(LSTSCI=SILT, time=selected_time, M=selected_M)
density_m_at_time = trim.RHO.isel(time=selected_time, M=selected_M) 
velocity_m_at_time = trim.velocity.isel(time=selected_time, M=selected_M) 

z_section_center = depth_center.isel(time=selected_time, M=selected_M) # density_m_31.depth_center.T #trim.depth_center.isel(time=this_time, M=this_M).T # why do i need to transpose this?
z_section_intf = trim.depth.isel(time=selected_time, M=selected_M) # why do i need to transpose this?

n_section = z_section_intf.YZ

_, mesh_N = np.meshgrid(trim.SIG_INTF.values, n_section)

2020-02-01T01:48:00.000000000


### Sanity check of dimensions

In [36]:
# # is there a way to quickly print a formatted table in jupyter without pandas?
# print('quad_NSigma\t', mesh_N.shape)
# print('n_section\t', n_section.shape)
# print('z_section_center', z_section_center.shape)
# print('z_section_intf\t', z_section_intf.shape)
# print('density_m_at_time', density_m_at_time.T.shape)

In [37]:
# sigma_percentages = np.diff(trim.SIG_LYR.values)*-100

In [38]:
bottom = z_section_intf.isel(SIG_INTF=-1)

In [39]:
legend_hack_intf = z_section_intf.isel(SIG_INTF=-1)
legend_hack_center = z_section_center.isel(SIG_LYR=-1)

In [24]:
plt.close("all")

## Plot Sigma interfaces and center with length

Fancy plot for report with inset to show detailed sigma-layers

In [1]:
# zscale = 20
fig, ax = plt.subplots(nrows=1, figsize=(10,4.5)) # scale to actual depth lenght: (20 ,4) for  factor 10
plt.suptitle('Vertical grid-section')
plt.title("Channel slope 1.25° - Basin slope 0.2°", fontsize=11)

not_all_centers = z_section_center.isel(SIG_LYR=slice(0,40))

ax.plot(n_section, z_section_intf, c='k', linewidth=0.55, marker='+', markersize=0.1)
ax.plot(n_section, not_all_centers, c='steelblue', linewidth=0.15, marker='+', markersize=0.1)

ax.xaxis.set_minor_locator(MultipleLocator(1000))
plt.xlim(0., 36200)

# hack to get only names in legend for lines
interfaces, = ax.plot(n_section, legend_hack_intf, c='k', linewidth=0.8, marker='+', markersize=0.)
centers, = ax.plot(n_section, legend_hack_center, c='steelblue', linewidth=0.8, marker='+', markersize=0.1)
# mark one line just because
# ax.plot(n_section, z_section_intf[:,42], c='hotpink', marker='o', markersize=1.5,)

# inset axes
axins = ax.inset_axes([0.5, 0.5, 0.47, 0.47])
axins.plot(n_section, z_section_intf, c='k',  linewidth=0.55, marker='+', markersize=0.1)
plt.setp(axins.spines.values(), color='tomato')
plt.setp([axins.get_xticklines(), axins.get_yticklines()], color='tomato')
plt.setp([axins.get_yticklines(), axins.get_yticklines()], color='tomato')
 
# sub region of the original image
x1, x2 = 13000, 17000
y1, y2 = -635, -600
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')

ax.indicate_inset_zoom(axins, edgecolor='tomato', alpha=0.5)

# ax.scatter(n_section, depth_m_31[:,10], s=1.5)
ax.set_xlabel('Length $n$ (m)', fontsize=13.5)
ax.set_ylabel('Depth (m)', fontsize=13.5)
ax.legend((interfaces, centers), ('$\sigma$ Interfaces', '$\sigma$ Centers'), fontsize="x-large", loc="lower left", markerscale=50)
fig.show()

NameError: name 'plt' is not defined

In [107]:
# fig_N, ax_N = plt.subplots(nrows=1, figsize=(9,6))
# ax_N.set_title('Sigma layers vertical section - N number')

# ax_N.plot(z_section_intf, c='k', linewidth=0.25, marker='+', markersize=0.1)
# ax_N.plot(z_section_center, c='g', linewidth=0.05, marker='+', markersize=0.1)
# ax_N.set_xlabel('N')
# ax_N.set_ylabel('Depth [m]')
# fig_N.show()

## Plot density cross-section

In [108]:
print(mesh_N[:,:].shape)
print(z_section_intf[:,:].shape)
print(density_m_at_time.T.shape)

(182, 81)
(182, 81)
(182, 80)


In [109]:
fig_vert, ax_vert2 = plt.subplots(nrows=1, figsize=(8,4))

ax_vert2.set_title(f'Density {scenario} {timestring}')
ax_vert2.set_ylim(-700, -200)
ax_vert2.set_xlim(2000, 12500)


mesh = ax_vert2.pcolormesh(mesh_N, z_section_intf, density_m_at_time.T, vmin=1025, vmax=1030)

# ax_vert2.plot(n_section, z_section_center, c='lightsteelblue', linewidth=0.10, alpha=0.5) #, marker='+', markersize=0.1)
ax_vert2.plot(n_section, z_section_intf[:,:], c='bisque', linewidth=0.08, alpha=0.5)


cbar = fig_vert.colorbar(mesh, ax=ax_vert2)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Density [kg/m3]', rotation=90)
cbar.ax.get_xaxis().labelpad = 15
ax_vert2.set_xlabel('Length $n$ [m]')
ax_vert2.set_ylabel('Depth [m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth [m]')

## Velocity cross section

In [150]:
fig_vert, ax_vert2 = plt.subplots(nrows=1, figsize=(8,4))

ax_vert2.set_title(f'Velocity {scenario} {timestring}')
ax_vert2.set_ylim(-550, -250)
ax_vert2.set_xlim(2000, 12500)

# or z_section.YZ both work
mesh = ax_vert2.pcolormesh(mesh_N, z_section_intf, velocity_m_at_time,
                          vmin=0, vmax=2.00, cmap='inferno') #, edgecolors='w', linewidths=0.1)

# ax_vert2.plot(n_section, z_section_center, c='lightsteelblue', linewidth=0.10, alpha=0.5) #, marker='+', markersize=0.1)
ax_vert2.plot(n_section, z_section_intf[:,:], c='bisque', linewidth=0.08, alpha=0.7)

# cbar = fig_vert.colorbar(mesh, orientation="horizontal", ax=ax_vert2)
# cbar.ax.get_yaxis().labelpad = 15
# cbar.ax.set_xlabel('Velocity [m/s]')
ax_vert2.set_xlabel('Length $n$ [m]')
# ax_vert2.set_ylabel('Depth [m]')

/Users/julesblom/Library/Caches/pypoetry/virtualenvs/delft3d-python-j5EFwY34-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Length $n$ [m]')

# Silt concentration

In [50]:
fig_vert, ax_vert2 = plt.subplots(nrows=1, figsize=(9,4))

ax_vert2.set_title(f'Silt concentration {scenario} {timestring}')
ax_vert2.set_ylim(-700, -200)

# or z_section.YZ both work
mesh = ax_vert2.pcolormesh(mesh_N, z_section_intf, concentration_silt.T, vmin=0, vmax=7, cmap='magma_r')
ax_vert2.plot(n_section, z_section_intf[:,:], c='dimgray', linewidth=0.08, alpha=0.5)

cbar = fig_vert.colorbar(mesh, ax=ax_vert2)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Silt concentration [%]',rotation=90)
ax_vert2.set_xlabel('Length $n$ [m]')
ax_vert2.set_ylabel('Depth [m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth [m]')

## Sand concentration

In [54]:
fig_vert, ax_vert2 = plt.subplots(nrows=1, figsize=(9,4))

ax_vert2.set_title(f'Sand concentration {scenario} {timestring}')
ax_vert2.set_ylim(-700, -200)

# or z_section.YZ both work
mesh = ax_vert2.pcolormesh(mesh_N, z_section_intf, concentration_sand.T, vmin=0, vmax=2, cmap='magma_r')
ax_vert2.plot(n_section, z_section_intf[:,:], c='dimgray', linewidth=0.08, alpha=0.5)

cbar = fig_vert.colorbar(mesh, ax=ax_vert2)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Sand concentration [%]',rotation=90)
ax_vert2.set_xlabel('Length $n$ [m]')
ax_vert2.set_ylabel('Depth [m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth [m]')

## Turbulent Energy

In [ ]:
fig_vert, ax_vert2 = plt.subplots(nrows=1, figsize=(9,4))

ax_vert2.set_title(f'Turbulent energy {scenario} {timestring}')
ax_vert2.set_ylim(-700, -300)
ax_vert2.set_xlim(0, 15000)


# or z_section.YZ both work
mesh = ax_vert2.pcolormesh(mesh_N, z_section_intf, turbu_energy_at_time.T, cmap='magma_r', vmin=0, vmax=0.05,)
ax_vert2.plot(n_section, z_section_intf[:,:], c='dimgray', linewidth=0.08, alpha=0.5)

cbar = fig_vert.colorbar(mesh, ax=ax_vert2)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Turbulent energy [%]',rotation=90)
ax_vert2.set_xlabel('Length $n$ [m]')
ax_vert2.set_ylabel('Depth [m]')

# Report BC plot

In [ ]:
# # jessy_hopman_uglyman = [(400, 500), (400 , -200)]

# fig_vert, ax_vert1 = plt.subplots(nrows=1, figsize=(9,6))

# ax_vert1.set_title('Vertical Section Density')
# ax_vert1.set_xlim(250, 2000)
# ax_vert1.set_ylim(-400, -200)

# # or z_section.YZ both work
# mesh = ax_vert1.pcolormesh(mesh_N, z_section_intf, density_m_at_time.T,
#                           vmin=1027, vmax=1035) #, edgecolors='w', linewidths=0.1)
# # ax_vert1.plot(n_section, z_section_center, c='lightsteelblue', linewidth=0.10, alpha=0.5) #, marker='+', markersize=0.1)
# ax_vert1.plot(n_section, z_section_intf[:,:], c='bisque', linewidth=0.1, alpha=1, marker='+', markersize=0.05)
# # interfaces, = ax.plot(n_section, legend_hack_intf, c='k', linewidth=0.8, marker='+', markersize=0.)

# ax_vert1.plot(n_section, z_section_intf[:,-53], c='white', marker='o', markersize=1.5,)


# ax_vert1.plot(n_section, z_section_intf[:,-53:None], c='white', linewidth=0.06,)
# sigma_text = ax_vert1.text(1000, -277, "$\sigma$-interface = 27", color="white", fontsize=13, rotation=-7.5)
# sigma_text.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='black')])


# ax_vert1.plot(n_section, z_section_intf[:,-2:None], c='black', linewidth=0.6,)
# sigma_text2 = ax_vert1.text(1000, -340, "$\sigma$-interface = 80", color="black", fontsize=13, rotation=-7.5)
# sigma_text2.set_path_effects([PathEffects.withStroke(linewidth=0.2, foreground='white')])


# ax_vert1.plot([255, 255], [-260, -305], c='coral', marker='o', linewidth=3,)

# dischage_text = ax_vert1.text(300, -300, "Discharge: 4500 m3/s", color="white", fontsize=11)
# discharge_text.set_path_effects([PathEffects.withStroke(linewidth=1, foreground='hotpink')])

# # ax_vert1.legend(interfaces, '$\sigma$ Interfaces', fontsize="x-large", loc="lower left", markerscale=50)

# cbar = fig_vert.colorbar(mesh, ax=ax_vert1)
# cbar.ax.get_yaxis().labelpad = 15
# cbar.ax.set_ylabel('Density [$kg/m^3$]', rotation=90, fontsize=13)
# ax_vert1.set_xlabel('Length $n$ [m]', fontsize=14)
# ax_vert1.set_ylabel('Depth [m]', fontsize=14)

In [ ]:
# plt.close("all")